In [1]:
from utils import *
from calc_config import *

In [2]:
abs_main, nucleus_name, pathfilename = pathfilename_gen(pcname,input_txt) ### Bug here, run once create folder once, flaw!!
os.chdir(abs_main)

## Reading the txt file and convert to a list object
text_whole = []
with open (pathfilename["source_text"], 'rt') as myfile:    
    for line in myfile:
        line = line.rstrip('\n')
        text_whole.append(line)

# Parameters extractions

In [3]:
## Extraction of num_particles data; Create a new class of object called num_orbitals
## Below the number of particles and orbitals all refer to the valence"s

num_particles = [None,None]

## Extract particles number information; Information within the lower bound and upper bound for pairing valence space
pattern_energy_start = "-------------------- Lower bound for Pairing Valence Space -------------------------"
pattern_energy_end   = "-------------------- Upper bound for Pairing Valence Space -------------------------"
energy_block_id = identify_line(text_whole, pattern_energy_start,pattern_energy_end);
energy_block_id.sort()

energy_neut_list = text_whole[energy_block_id[0]+1: energy_block_id[1]]
energy_prot_list = text_whole[energy_block_id[2]+1: energy_block_id[3]]
energy_neut_list = remove_line(energy_neut_list,"Fermi")
energy_prot_list = remove_line(energy_prot_list,"Fermi")

num_neut_list = []; num_prot_list = [] ## number of nucleons
for line in energy_neut_list:
    num_neut_list.append(int(line[53:55]))
for line in energy_prot_list:
    num_prot_list.append(int(line[53:54]))

### Summary of this Block
num_particles[0] = sum(num_neut_list); 
num_particles[1] = sum(num_prot_list); 
num_particles = tuple(num_particles)
### SUMMARY END ###

### Getting the hf_lvl index for the lower bound hf states
n_start = int(energy_neut_list[0][7:10].strip())
p_start = int(energy_prot_list[0][7:10].strip())

In [4]:
## Drop the information of the unbound/continuum states, 
## for example single particle states with positive energies.
unbounded_neut_start = 100; unbounded_prot_start = 100 # state with hf_lvl larger than this will be eliminated
for counter, line in enumerate(energy_neut_list):
    line = line.replace(')',' ')
    line = remove_character(line,'()')
    split_list = line.split()
    dummy_epsilon = float(split_list[4])
    if dummy_epsilon > 0:
        unbounded_neut_start = int(split_list[1])
        del energy_neut_list[counter:]

for counter, line in enumerate(energy_prot_list):
    line = line.replace(')',' ')
    line = remove_character(line,'()')
    split_list = line.split()
    dummy_epsilon = float(split_list[4])
    if dummy_epsilon > 0:
        unbounded_prot_start = int(split_list[1])
        del energy_prot_list[counter:]


num_orbitals = [None,None]
num_spin_orbitals = None

## from the above we got the orbitals information; Summary as follows
num_neut_orbitals = len(energy_neut_list); num_prot_orbitals = len(energy_prot_list)
num_orbitals[0] = num_neut_orbitals; num_orbitals[1] = num_prot_orbitals; num_orbitals = tuple(num_orbitals)
num_spin_orbitals = sum(num_orbitals)

# Twobody Matrix Terms source

In [5]:
## Extract twobody matrix elements information
extracted_twobody = []
pattern_hf = re.compile("HF INDEX AND ISOSPIN")
for line in text_whole:
    if pattern_hf.search(line) != None:      # If a match is found
        extracted_twobody.append((line.rstrip('\n')).lstrip('HF INDEX AND ISOSPIN'))
### Store the information extracted into a dataframe; and output them as csv for later viewing/assessment
obs_twobody_df = pd.DataFrame(columns = ["q_i1","q_i2","q_f1","q_f2","V_ffii","hf_lvl_i1","iso_i1","hf_lvl_i2","iso_i2","hf_lvl_f1","iso_f1","hf_lvl_f2","iso_f2"])
for counter, extracted_twobody_row in enumerate(extracted_twobody):
    row_split = extracted_twobody_row.split()
    dummy_q_i1 = np.nan; dummy_q_i2 = np.nan; dummy_q_f2 = np.nan; dummy_q_f2 = np.nan; #q is qubit; i1,i2,f1,and f2  qubit indeces
    dummy_V_ffii = float(row_split[8])
    dummy_hfl_i1 = int(row_split[0]); dummy_iso_i1 = int(row_split[1])
    dummy_hfl_i2 = int(row_split[2]); dummy_iso_i2 = int(row_split[3])
    dummy_hfl_f1 = int(row_split[4]); dummy_iso_f1 = int(row_split[5])
    dummy_hfl_f2 = int(row_split[6]); dummy_iso_f2 = int(row_split[7])
    
    ## Skip appending interations involving unbounded states
    # remove unbound neutrons
    if (((dummy_iso_i1 == 1) and (dummy_hfl_i1 >= unbounded_neut_start)) or
        ((dummy_iso_i2 == 1) and (dummy_hfl_i2 >= unbounded_neut_start)) or
        ((dummy_iso_f1 == 1) and (dummy_hfl_f1 >= unbounded_neut_start)) or
        ((dummy_iso_f2 == 1) and (dummy_hfl_f2 >= unbounded_neut_start))):
        continue
    # remove unbound protons:
    if (((dummy_iso_i1 == 2) and (dummy_hfl_i1 >= unbounded_prot_start)) or
        ((dummy_iso_i2 == 2) and (dummy_hfl_i2 >= unbounded_prot_start)) or
        ((dummy_iso_f1 == 2) and (dummy_hfl_f1 >= unbounded_prot_start)) or
        ((dummy_iso_f2 == 2) and (dummy_hfl_f2 >= unbounded_prot_start))):
        continue
    
    
    appending_row = [
        dummy_q_i1,dummy_q_i2,dummy_q_f2,dummy_q_f2,dummy_V_ffii,
        dummy_hfl_i1,dummy_iso_i1,
        dummy_hfl_i2,dummy_iso_i2,
        dummy_hfl_f1,dummy_iso_f1,
        dummy_hfl_f2,dummy_iso_f2]
    obs_twobody_df.loc[counter] = appending_row

### Define the qubit indeces
#### Start #### create to make the code cleaner, to define the row name for the dataframe and to use them to manipulate the dataframe
suffix_list = ["i1","i2","f1","f2"]; iso_list=[]; q_list=[]; hf_lvl_list=[]
for suffix in suffix_list:
    iso_list.append("iso_"+suffix); q_list.append("q_"+suffix); hf_lvl_list.append("hf_lvl_"+suffix)
#### End ####
for index, row in obs_twobody_df.iterrows():
    for (iso_dum,q_dum,hf_lvl_dum) in zip(iso_list, q_list, hf_lvl_list):
        if row[iso_dum] == 1:         ## iso = 1 represents neutron; 2 represents proton
            row[q_dum] = row[hf_lvl_dum] - n_start
        elif row[iso_dum] == 2:
            row[q_dum] = row[hf_lvl_dum] - p_start + num_neut_orbitals

### To csv
obs_twobody_df.reset_index(drop=True,inplace=True)
obs_twobody_df.to_csv(pathfilename["output_2B-source_csv"])

# Onebody Matrix Terms source

In [6]:
# Extract onebody information
## Store the onebody terms into panda dataframes and export as csv
obs_onebody_df = pd.DataFrame(columns = ["q_i","spin","epsilon","hf_lvl","iso"])

for counter, line in enumerate(energy_neut_list+energy_prot_list):
    dummy_spin = None
    dummy_iso = None
    dummy_df_row = []
    line = line.replace(')',' ')
    line = remove_character(line,'()')
    split_list = line.split()
    if int(split_list[6]) == 1:
        dummy_iso = 1
    elif int(split_list[6]) == -1:
        dummy_iso = 2
    else: 
        pass

    if  float(Fraction(split_list[2])) < 0:
        dummy_spin = -1
    elif float(Fraction(split_list[2])) > 0:
        dummy_spin = 1
    else:
        pass
    dummy_hf_lvl = int(split_list[1])
    dummy_epsilon = float(split_list[4])
    dummy_df_row = [np.nan,dummy_spin,dummy_epsilon,dummy_hf_lvl,dummy_iso]
    obs_onebody_df.loc[counter] = dummy_df_row
obs_onebody_source_df_output = obs_onebody_df.copy() ## clone the df so that the below codes doesnt mess things up(action is debatable, but i dont care)

### Define the qubit indeces for obs_onebody_df (conver hf_lvl to q_i; different encoding/indexing)
for index, row in obs_onebody_source_df_output.iterrows():
    if row["iso"] == 1:
        row["q_i"] = row["hf_lvl"] - n_start
    elif row["iso"] == 2:
        row["q_i"] = row["hf_lvl"] - p_start + num_orbitals[0]

### To csv
obs_onebody_source_df_output.to_csv(pathfilename['output_1B-source_csv'])
obs_onebody_df = obs_onebody_source_df_output # Change the df back to the changed values

# Filter out unwanted interactions from obs_twobody_df
# Create obs_twobody_H_input_df_output (input for VQE)

In [7]:
## Removal of unbound states wont affect anything here, as the following takes the input of the num_orbitals and num_spin_orbital
from itertools import combinations
## Create interaction of Interest aka promotion_list
### Allowed pair combinations
# non_repeat_list = list(combinations(range(0,num_spin_orbitals),2))    ## List pair combination that doesn't repeat same orbitals (particle doesnt occupy same orbital twice)
degenerate_levels_list = degenerate_pair_gen(num_orbitals)      ## Degenerate level list, list of pair combinations that share the same energy
spin_pair_list = antipara_spin_pair_gen(obs_onebody_df,num_orbitals)  ## list of Pairs with T=1 (include Tz=0,+-1)
init_pair_list = HFground_pair_list(num_particles, num_orbitals)      ## Allowed Initial state
## to allow for set that include blocked levels for Be9, and set that cater for promotions in Be10

neut_state_init = list(range(0,num_particles[0]))
prot_state_init = list(range(num_orbitals[0],num_orbitals[0]+num_particles[1]))
occupied_state = neut_state_init + prot_state_init

allowed_init = list(set(degenerate_levels_list) & set(init_pair_list) & set(spin_pair_list))
allowed_fina = [] ; all_fina = list(set(degenerate_levels_list) & set(spin_pair_list))
for fina in all_fina: 
    if (fina[0] in occupied_state) or (fina[1] in occupied_state):
        pass
    else:
        allowed_fina.append(fina)

### Allowed promotions
#### Neutron orbitals (indeces assigned to qubit)
neut_orbitals_list = list(range(0,num_orbitals[0]))
#### Proton orbitals (indeces assigned to qubit)
prot_orbitals_list = list(range(num_orbitals[0], sum(num_orbitals)))
promotion_list = []
for init in allowed_init:
    promotion_list.append((init,init))
    for fina in allowed_fina:
        if ((init[0] in neut_orbitals_list) == (fina[0] in neut_orbitals_list) and
            (init[1] in neut_orbitals_list) == (fina[1] in neut_orbitals_list) and
            (init[0] in prot_orbitals_list) == (fina[0] in prot_orbitals_list) and
            (init[1] in prot_orbitals_list) == (fina[1] in prot_orbitals_list)):
            excitations = [init,fina]; excitations.sort();
            promotion_list.append(tuple(excitations) if (excitations not in promotion_list) else tuple())
promotion_list.sort()

In [14]:
allowed_init

[(0, 1), (0, 8), (1, 9), (8, 9)]

In [9]:
## create a drop_list that containes row index to drop
drop_list = []
for line_number, (index, row) in enumerate(obs_twobody_df.iterrows()):
    init_1 = int(row['q_i1']); init_2 = int(row['q_i2']);
    fina_1 = int(row['q_f1']); fina_2 = int(row['q_f2']);
    current_promotion = ((init_1,init_2),(fina_1,fina_2))
    if (current_promotion in promotion_list):
        pass
    else:
        drop_list.append(line_number)

obs_twobody_df.drop(drop_list,axis=0,inplace=True)
obs_twobody_df.reset_index(drop=True,inplace=True)

### To csv
obs_twobody_H_input_df_output = obs_twobody_df.copy()
obs_twobody_H_input_df_output.to_csv(pathfilename['output_2B-H_input_csv'])

# Create obs_onebody_H_input_df_output (input for VQE)

In [10]:
##### obs_onebody_df will always be the name of being_worked_df of onebody
del obs_onebody_df
obs_onebody_df = pd.DataFrame(columns =['q_i','q_f','delta'])

### init_fina_list
if_list = sin_bod_if_list_gen(num_orbitals)
epsilon_list = obs_onebody_source_df_output.loc[:,"epsilon"]

for counter, if_ in enumerate(if_list):
    delta = epsilon_list[if_[1]] - epsilon_list[if_[0]]
    obs_onebody_df.loc[counter] = [if_[0], if_[1], round(delta,6)]

obs_onebody_H_input_df_output = obs_onebody_df.copy()
obs_onebody_H_input_df_output.to_csv(pathfilename['output_1B-H_input_csv'])

In [11]:
# %store promotion_list
# %store num_orbitals
# %store num_particles
# %store num_spin_orbitals
# %store obs_onebody_df
# %store obs_twobody_df
# %store abs_main
# %store pathfilename
# %store nucleus_name